In [1]:
import tkinter as tk
from tkinter import messagebox
import time

# --------------------------- GAME LOGIC ---------------------------

class QuanGame:
    def __init__(self):
        # Initialize game state
        self.o = [0] * 13  # 1-12
        self.q = [False] * 13
        self.q[6] = True
        self.q[12] = True
        self.quan = [None] * 13  # Labels for special pits
        self.gold_p1 = 0
        self.gold_p2 = 0
        self.time_bh = 0
        self.time_limit = 0
        self.root = None
        self.gui_elements = {}
        self.oke = False  # Flag to indicate if the game has started
        self.current_player = 'player1'  # 'player1' or 'player2'

    # --------------------------- GUI FUNCTIONS ---------------------------

    def o_r(self, vt):
        """Read the value of pit vt."""
        return self.o[vt]

    def o_w(self, vt, gt):
        """Write the value gt to pit vt."""
        self.o[vt] = gt
        self.gui_elements['button'][vt].config(text='' if gt == 0 else str(gt))

    def showw(self, me, w):
        """Update the status label."""
        self.gui_elements['show_tt'].config(text=f"{me}: {w}")

    def luot(self, type):
        """Update turn indicators."""
        if type == 'player1':
            self.gui_elements['lb_var'].config(state='normal')
            self.gui_elements['lt_var'].config(state='disabled')
            self.gui_elements['lb_var'].var.set(1)
            self.gui_elements['lt_var'].var.set(0)
        elif type == 'player2':
            self.gui_elements['lb_var'].config(state='disabled')
            self.gui_elements['lt_var'].config(state='normal')
            self.gui_elements['lb_var'].var.set(0)
            self.gui_elements['lt_var'].var.set(1)

    def get_level(self):
        """Get the selected difficulty level."""
        val = self.gui_elements['difficulty'].get()
        if val == 1:
            return 30
        elif val == 2:
            return 25
        elif val == 3:
            return 15
        elif val == 4:
            return 5
        else:
            return 25  # Default value

    def get_sec(self):
        """Get current time in seconds since epoch."""
        return int(time.time())

    def tinh_thoi_gian(self):
        """Calculate remaining time."""
        conlai = self.time_limit - (self.get_sec() - self.time_bh)
        self.gui_elements['stime'].config(text=f"Thời gian còn lại: {conlai}")
        if conlai <= 0:
            return False
        return True

    def chon_huong(self, o_so):
        """Choose direction for the move."""
        self.showw('', '')
        direction = None

        def select_direction(dir):
            nonlocal direction
            direction = dir
            self.gui_huong.destroy()

        self.gui_huong = tk.Toplevel(self.root)
        self.gui_huong.title(f"Bạn chọn ô số {o_so}!")
        tk.Label(self.gui_huong, text=f"Hãy chọn hướng đi cho ô số {o_so}").pack(pady=10)
        if o_so < 6:
            tk.Button(self.gui_huong, text=">>>", command=lambda: select_direction(1)).pack(side=tk.RIGHT, padx=20, pady=20)
            tk.Button(self.gui_huong, text="<<<", command=lambda: select_direction(-1)).pack(side=tk.LEFT, padx=20, pady=20)
        else:
            tk.Button(self.gui_huong, text=">>>", command=lambda: select_direction(-1)).pack(side=tk.RIGHT, padx=20, pady=20)
            tk.Button(self.gui_huong, text="<<<", command=lambda: select_direction(1)).pack(side=tk.LEFT, padx=20, pady=20)

        self.root.wait_window(self.gui_huong)
        return direction

    def xoay_vong(self, vt, huongdi, so_o=12):
        """Rotate around the pits."""
        vt += huongdi
        if vt <= 0:
            return so_o
        if vt > so_o:
            return 1
        return vt

    def process(self, o_so):
        """Process player's move."""
        if self.o_r(o_so) > 0:
            current_player = self.current_player
            huongdi = self.chon_huong(o_so)
            if huongdi is None:
                return  # Player closed the direction selection window
            print(f"{current_player} selected pit {o_so} with direction {huongdi}")
            earned = self.buoc_di(current_player, o_so, huongdi)
            if current_player == 'player1':
                self.gold_p1 += earned
                self.gui_elements['s_p1'].config(text=f"Player 1: {self.gold_p1} !")
            else:
                self.gold_p2 += earned
                self.gui_elements['s_p2'].config(text=f"Player 2: {self.gold_p2} !")
            self.check_o()
            # Switch to the other player
            if self.current_player == 'player1':
                self.current_player = 'player2'
                self.luot('player2')
            else:
                self.current_player = 'player1'
                self.luot('player1')
        else:
            messagebox.showerror("Lỗi", "Bạn không thể chọn ô không có tiền")

    def buoc_di(self, me, o_so, huongdi):
        """Execute a move from pit o_so in direction huongdi."""
        giatri = self.o_r(o_so)
        hientai = o_so
        self.o_w(hientai, 0)
        an = 0
        while True:
            hientai = self.xoay_vong(hientai, huongdi)
            self.showw(me, f"Đang ở ô {hientai}, số quân đang dải {giatri}")
            current = self.o_r(hientai)
            self.o_w(hientai, current + 1)
            giatri -= 1
            if giatri <= 0:
                hientai = self.xoay_vong(hientai, huongdi)
                if self.o_r(hientai) == 0:
                    while self.o_r(hientai) == 0:
                        hientai = self.xoay_vong(hientai, huongdi)
                        if self.o_r(hientai) == 0:
                            break
                        an += self.o_r(hientai)
                        self.showw(me, f'Ăn được {self.o_r(hientai)} quân tại ô {hientai}')
                        self.o_w(hientai, 0)
                        hientai = self.xoay_vong(hientai, huongdi)
                        self.root.update()
                        time.sleep(0.5)
                    break
                else:
                    if hientai in [6, 12]:
                        if self.q[hientai] and (self.o_r(hientai) - 10 <= 0):
                            break
                        elif self.o_r(hientai) <= 1:
                            break
                        else:
                            giatri = 1
                            self.o_w(hientai, self.o_r(hientai) - 1)
                    else:
                        giatri = self.o_r(hientai)
                        self.o_w(hientai, 0)
            self.root.update()
            time.sleep(0.5)
        if an > 0:
            self.showw(me, f'Ăn được {an} quân!')
        else:
            self.showw(me, 'Chẫng!')
        return an

    def check_o(self):
        """Check the game state after each move."""
        all_empty = all(self.o_r(i) == 0 for i in range(1, 13))
        if all_empty:
            if self.gold_p1 > self.gold_p2:
                messagebox.showinfo("Thông báo", "Xin chúc mừng Player 1 là người thắng cuộc :))!")
            elif self.gold_p1 == self.gold_p2:
                messagebox.showinfo("Thông báo", "Haizz, trận này hoà cmnr hihi :))!")
            else:
                messagebox.showinfo("Thông báo", "Xin chúc mừng Player 2 là người thắng cuộc :))!")
            self.root.destroy()
            exit()
        # Check if player1 pits are empty
        if all(self.o_r(i) == 0 for i in range(1, 6)):
            if self.gold_p1 >= 5:
                self.showw('Player 1', 'đã hết quân, đã dải mỗi ô 1 quân!')
                for i in range(1, 6):
                    self.o_w(i, 1)
                    self.gold_p1 -= 1
                    self.gui_elements['s_p1'].config(text=f"Player 1: {self.gold_p1} !")
                    self.root.update()
                    time.sleep(0.5)
            else:
                messagebox.showinfo("Thông báo", "Player 1 không đủ tiền để dải quân, Player 1 THUA :))")
                self.root.destroy()
                exit()
        # Check if player2 pits are empty
        if all(self.o_r(i) == 0 for i in range(7, 12)):
            if self.gold_p2 >= 5:
                self.showw('Player 2', 'đã hết quân, đã dải mỗi ô 1 quân!')
                for i in range(7, 12):
                    self.o_w(i, 1)
                    self.gold_p2 -= 1
                    self.gui_elements['s_p2'].config(text=f"Player 2: {self.gold_p2} !")
                    self.root.update()
                    time.sleep(0.5)
            else:
                messagebox.showinfo("Thông báo", "Player 2 không đủ tiền để dải quân, Player 2 THUA :))")
                self.root.destroy()
                exit()

    # --------------------------- RUN GAME ---------------------------

    def main(self):
        """Initialize and run the main game window."""
        self.root = tk.Tk()
        self.root.title("Ô Ăn Quan QT - Two Players")

        # Create buttons for pits
        self.gui_elements['button'] = {}
        positions = {
            10: (216, 56),
            8: (312, 56),
            9: (264, 56),
            7: (360, 56),
            4: (312, 104),
            3: (264, 104),
            11: (168, 56),
            1: (168, 104),
            5: (360, 104),
            2: (216, 104),
            6: (416, 56),
            12: (120, 56)
        }
        for key, (x, y) in positions.items():
            if key in [6, 12]:
                initial = "0"  # Set the special pits to start with 0 seeds
                self.o[key] = 0
            else:
                initial = "5"
                self.o[key] = 5
            btn = tk.Button(self.root, text=initial, width=5, height=2,
                            command=lambda k=key: self.on_button_click(k))
            if key in [6, 12]:  # Adjust size for special pits
                btn.config(width=4, height=5)  # Example sizes, adjust as needed
            btn.place(x=x, y=y)
            self.gui_elements['button'][key] = btn

        # Labels for pits numbers
        for i in range(1, 6):
            tk.Label(self.root, text=str(i)).place(x=184 + 48*(i-1), y=144)

        # Labels for special pits (6 and 12)
        self.quan[6] = tk.Label(self.root, text="(Q)" if self.q[6] else "")
        self.quan[6].place(x=464, y=88)
        self.quan[12] = tk.Label(self.root, text="(Q)" if self.q[12] else "")
        self.quan[12].place(x=96, y=88)

        # Radio buttons for difficulty (if still needed)
        self.gui_elements['difficulty'] = tk.IntVar(value=1)  # Default to "Dễ"

        tk.Radiobutton(self.root, text="Dễ", variable=self.gui_elements['difficulty'], value=1).place(x=504, y=40)
        tk.Radiobutton(self.root, text="Trung bình", variable=self.gui_elements['difficulty'], value=2).place(x=504, y=64)
        tk.Radiobutton(self.root, text="Khó", variable=self.gui_elements['difficulty'], value=3).place(x=504, y=88)
        tk.Radiobutton(self.root, text="Ác mộng", variable=self.gui_elements['difficulty'], value=4).place(x=504, y=112)

        # Checkbox for random
        self.gui_elements['rand'] = tk.IntVar()
        tk.Checkbutton(self.root, text="Random", variable=self.gui_elements['rand']).place(x=504, y=136)

        # Start button
        self.gui_elements['start'] = tk.Button(self.root, text="Chơi", command=self.start_game)
        self.gui_elements['start'].place(x=504, y=176)

        # Labels for gold
        self.gui_elements['s_p2'] = tk.Label(self.root, text=f"Player 2: {self.gold_p2} !")
        self.gui_elements['s_p2'].place(x=8, y=72)
        self.gui_elements['s_p1'] = tk.Label(self.root, text=f"Player 1: {self.gold_p1} !")
        self.gui_elements['s_p1'].place(x=8, y=136)

        # Turn indicators
        self.gui_elements['lb_var'] = tk.Checkbutton(self.root, text="Player 1's turn", state='disabled')
        self.gui_elements['lb_var'].var = tk.IntVar()
        self.gui_elements['lb_var'].config(variable=self.gui_elements['lb_var'].var)
        self.gui_elements['lb_var'].place(x=88, y=184)

        self.gui_elements['lt_var'] = tk.Checkbutton(self.root, text="Player 2's turn", state='disabled')
        self.gui_elements['lt_var'].var = tk.IntVar()
        self.gui_elements['lt_var'].config(variable=self.gui_elements['lt_var'].var)
        self.gui_elements['lt_var'].place(x=88, y=8)

        # Time remaining
        self.gui_elements['stime'] = tk.Label(self.root, text="")
        self.gui_elements['stime'].place(x=224, y=184)

        # Show status
        self.gui_elements['show_tt'] = tk.Label(self.root, text="")
        self.gui_elements['show_tt'].place(x=256, y=8)

        self.root.geometry("586x232+269+232")
        self.root.resizable(False, False)
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()


    def start_game(self):
        """Start the game when the 'Chơi' button is clicked."""
        messagebox.showinfo("Thông báo", "Player 1 bắt đầu, hãy chú ý thời gian nhé!")
        self.oke = True
        self.time_limit = self.get_level()
        self.time_bh = self.get_sec()
        # Delete the start button
        self.gui_elements['start'].destroy()
        # Indicate it's Player1's turn
        self.luot('player1')

    def on_close(self):
        """Handle window close event."""
        self.root.destroy()
        exit()

    def on_button_click(self, key):
        """Handle pit button clicks."""
        if self.oke:
            # Determine which player owns the pit
            if key in range(1, 6) and self.current_player == 'player1':
                self.process(key)
            elif key in range(7, 12) and self.current_player == 'player2':
                self.process(key)
            else:
                messagebox.showerror("Lỗi", "Không phải lượt của bạn hoặc ô không thuộc về bạn!")

# --------------------------- RUN GAME ---------------------------

if __name__ == "__main__":
    game = QuanGame()
    game.main()


player1 selected pit 3 with direction -1
player2 selected pit 11 with direction -1
